# Autocomplete

In this section, we will explore the autocomplete algorithm. Autocomplete is a feature commonly used in applications, providing word or phrase suggestions as users type. We will be working on implementing this functionality using a data structure called a Trie, as seen on class.

To get started, we will load a dataset that will serve as the basis for your autocomplete system. We will use the Hugging Face `datasets` library. **Hugging Face** is a leading platform for NLP and Machine Learning that provides access to state-of-the-art models, datasets, and tools for developing and deploying applications in the field of natural language understanding. Familiarizing yourself with this platform is valuable if you're interested in NLP and ML.


Before we can load the dataset, we need to install the `datasets` library. The following command will install it: `!pip install datasets -q`

In [1]:
# install the necessary library
!pip install datasets -q

In [2]:
# import and load the dataset
from datasets import load_dataset
import string
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
ds = load_dataset("google_wellformed_query")

In [3]:
# explore ds properties
print(ds.keys())
print(len(ds['train']))
print(len(ds['validation']))
print(len(ds['test']))
print(ds['train'][:2])

dict_keys(['train', 'test', 'validation'])
17500
3750
3850
{'rating': [0.20000000298023224, 0.4000000059604645], 'content': ['The European Union includes how many ?', 'What are Mia Hamms accomplishment ?']}


## Question 4 (5 points):

Write a function `read_data` to read and preprocess the data.

We'll use a dataset called `google_wellformed_query`. Google's query wellformedness dataset was created by crowdsourcing well-formedness annotations for 25,100 queries from the Paralex corpus. Each query in this dataset has been annotated by five raters, each providing a rating of 1 or 0 to indicate whether or not the query is well-formed. A rating of 1 suggests that the query is well-formed, while a rating of 0 suggests otherwise.

Note: For this assignment, you only need to work with the `train` split of the dataset.

In [4]:
def read_data(dataset, rate):
    """
    This function reads data from a dataset and filters it based on a specified rating threshold.

    Args:
        dataset (dict): The dataset to read from.
        rate (float): The rating threshold. Sentences with a rating greater than or equal to this threshold will be included in the output.

    Returns:
        list: A list of the sentences.
    """
    sentences = [sentence['content'] for sentence in ds['train'] if sentence['rating'] >= rate]
    return sentences

In [5]:
# Read the data using the dataset library and your function here:
# Enter desired rate
rate = 0.7
sentences = read_data(ds,rate)

## Question 5 (15 points):

Complete the Autocomplete function we saw in class, which provides automatic word completions as follows:

*   The function should accept user input as a complete sentence.
*   It will simulate word-by-word printing.
*   Suggestions will be based on probability calculations within a Trie data structure.
*   Probabilities are determined by counters, representing word occurrences in the Trie.

**Example:**

User types **"How do you make the perfect pizza?"**, your function returns the following:
*   **How** many calories in a cup of bluberries
*   **How do** you change the alternator belt on a 1998 audi a4
*   **How do you** make the perfect pizza?
*   **How do you make** the perfect pizza?
*   **How do you make the** perfect pizza?
*   **How do you make the perfect** pizza?
*   **How do you make the perfect pizza?**

**Note:** This is an example based on a specific dataset. Changing or expanding the dataset could potentially yield better results.

**Feel free to make changes and adjustments to the code.**

In [6]:
class TrieNode(object):
    """
    Our trie node implementation. Very basic, but does the job.
      word: The word associated with the node.
      children: A list of child nodes.
      sentence_finished: A boolean indicating if the current node marks the end of a sentence.
      counter: A count of how many times the word associated with this node has appeared in the addition process.
    """
    def __init__(self, word):
        self.word = word
        self.children = []
        # Is it the last word of the sentence.
        self.sentence_finished = False
        # How many times this word appeared in the addition process
        self.counter = 1

class Trie():
    def __init__(self, text):
        """
        Init a Trie with the given sentences list.
        During initialization, it removes punctuation, converts sentences to lowercase, and adds them to the Trie using the add method.
        """
        self.root = TrieNode('*')
        self.text = text
        self.punctuation = set(string.punctuation)
        for sentence in self.text:
            sentence = ''.join(char for char in sentence if char not in self.punctuation)
            self.add(sentence.lower())

    def add(self, sentence):
        """
        This method adds a sentence to the Trie structure.
        It breaks the sentence into words, and for each word, it traverses the Trie, adding new nodes as necessary or updating existing nodes.
        """
        node = self.root
        # root == *
        for word in sentence.split():
            found_in_child = False
            # Search for the word in the children of the present `node`
            for child in node.children:
                if child.word == word:
                    # We found it, increase the counter by 1 to keep track that another word has it as well
                    child.counter += 1
                    # And point the node to the child that contains this char
                    node = child
                    found_in_child = True
                    break
            # We did not find it so add a new chlid
            if not found_in_child:
                new_node = TrieNode(word)
                node.children.append(new_node)
                # And then point node to the new child
                node = new_node
        # Everything finished. Mark it as the end of a word.
        node.sentence_finished = True

    def find_prefix(self, prefix):
        """
        Check and return
          1. If the prefix exsists in any of the words we added so far
          2. If yes then how many words actually have the prefix
          3. The node where the prefix ends in the Trie structure
        """
        node = self.root
        # If the root node has no children, then return False.
        # Because it means we are trying to search in an empty trie
        if not self.root.children:
            return False, 0, node

        for word in prefix.split():
            word_not_found = True
            # Search through all the children of the present `node`
            for child in node.children:
                if child.word == word:
                    # We found the word existing in the child.
                    word_not_found = False
                    # Assign node as the child containing the word and break
                    node = child
                    break
            # Return False anyway when we did not find a word.
            if word_not_found:
                return False, 0, node
        # Well, we are here means we have found the prefix. Return true to indicate that
        # And also the counter of the last node. This indicates how many words have this
        # prefix
        return True, node.counter, node

    def list_all_children(self, prefix):
        """
        Returns all the children of the node at the end of a prefix, with the counter for each child
        """
        node = self.root
        auto_complete_sentence = prefix
        (exists, counter, node) = self.find_prefix(auto_complete_sentence)
        prefixes = []
        if exists:
          for child in node.children:
            prefixes.append((child.word, child.counter))

        return prefixes

    def auto_complete(self, prefix):
        """
        Suggests the most probable word completion for a given prefix.

        Args:
        - prefix (str): The input prefix to autocomplete.

        Returns:
        - suggestion (str): The suggested word completion.
        """
        sentence = prefix
        suggestion_options = self.list_all_children(sentence)
        suggestion = ''
        while suggestion_options:
          # find most appeared word in children
          max_word = max(suggestion_options, key=lambda x: x[1])
          suggestion += ' ' + max_word[0]
          suggestion_options = self.list_all_children(sentence + suggestion)
        return suggestion

    def display_autocomplete(self, user_input):
        # Display autocomplete suggestion with HTML formatting
        suggestion = self.auto_complete(self._clean_input(user_input))
        html_output = f'{user_input} <span style="color: #87CEEB;">{suggestion}</span>'
        display(HTML(html_output))

    def _clean_input(self, text):
      return ''.join(char for char in text.lower() if char not in set(string.punctuation))

    def init_auto_complete(self):
        """
        Initializes the auto-complete system and allows the user to interactively
        input prefixes and get suggestions.

        """
        print("Type here (press 'Ctrl+M+I' to exit):")

        try:
            while True:
                user_input = input()
                func_input = ''
                for word in user_input.split():
                  func_input += ' ' + word
                  self.display_autocomplete(func_input)
        except KeyboardInterrupt:
            print("\nExiting auto-complete system.")



In [7]:
# Example:
t = Trie(sentences)

t.init_auto_complete()

Type here (press 'Ctrl+M+I' to exit):



Exiting auto-complete system.
